In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)

df = pd.read_csv('~/data/domain_stop_removed_bias_text.csv')

df.head()

text_label = df[['text_stop_removed', 'audience_combined_bias']]

def replace_str(x):
    if x == 'left':
        return 0
    elif x == 'center':
        return 1
    else:
        return 2

text_label['label'] = text_label['audience_combined_bias'].apply(lambda x: replace_str(x))

text_label = text_label[['text_stop_removed', 'label']]

text_label.columns = ['text', 'label']

text_label['text'] = text_label['text'].astype('string')

<ipython-input-1-7773746b0030>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_label['label'] = text_label['audience_combined_bias'].apply(lambda x: replace_str(x))


In [2]:
from sklearn.model_selection import train_test_split

In [5]:
text_label

,text,label
0,story update department health human service h...,0
1,story produce originally publish grist reprodu...,0
2,cúcuta colombia close military intervention ve...,0
3,glisten tiara royal purple line sash burroughs...,0
4,washington reuters democrat house representati...,1
...,...,...
148678,adam jaren angel john sebastian star anthony b...,0
148679,washington reuters america policy political ac...,1
148680,tuesday broadcast msnbc beat filmmaker michael...,2
148681,50 female ambassador administration protect yo...,0


In [7]:
x = text_label['text']
y = text_label['label']

In [8]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.2, stratify=y)

In [11]:
train_df = pd.concat([train_x, train_y], axis=1)

In [13]:
train_sample = train_df.groupby('label').sample(10000)

In [15]:
train_sample['label'].value_counts()

0    10000
1    10000
2    10000
Name: label, dtype: int64

In [19]:
train_sample

,text,label
11812,washington house republican wednesday quietly ...,0
146684,core hugh edmeades salesman albeit dynamic ent...,0
89753,people injure explosion south florida strip ma...,0
95864,50-ton canister fill nuclear waste wedge insid...,0
58461,town hall sunday mayor pete buttigieg confront...,0
...,...,...
93363,rep alexandria ocasio cortez ny friday deny pl...,2
43140,guilty great president democrat president dona...,2
54856,feel heart race leg stay seat end butterfly st...,2
121515,sen bernie sander vt shoot sen kamala harris s...,2


In [22]:
from simpletransformers.classification import ClassificationModel


# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=3)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [23]:
model.train_model(train_sample, output_dir="ouputs/")

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/3750 [00:00<?, ?it/s]

/opt/conda/envs/st/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(3750, 0.6594880789970359)

In [24]:
test_df = pd.concat([test_x, test_y], axis=1)

In [25]:
test_df

,text,label
12039,line hour woody grind presidential compound mo...,0
42961,describe kind real process engage recently hug...,0
93026,brawl break rise garden thursday president tru...,2
92253,seren steve sinisi livestock farm old crow ran...,0
10036,generational change guard social action decade...,0
...,...,...
121411,washington reuters senior state department war...,1
80254,andrew weissmann prosecutor s mueller iii russ...,0
106651,cnn dalton cottrell cheyenne hedrick iowa hone...,0
35242,supreme court reject gay death row inmate appe...,0


In [29]:

from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

In [27]:
test_sample = test_df.groupby('label').sample(2000)

In [28]:
test_sample['label'].value_counts()

0    2000
1    2000
2    2000
Name: label, dtype: int64

In [30]:
result, model_outputs, wrong_predictions = model.eval_model(test_sample, f1=f1_multiclass, acc=accuracy_score)

/opt/conda/envs/st/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:1025: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Running Evaluation:   0%|          | 0/750 [00:00<?, ?it/s]

In [31]:
result

{'mcc': 0.7429227735545573,
 'f1': 0.8281666666666667,
 'acc': 0.8281666666666667,
 'eval_loss': 0.4464303920306265}

In [32]:
model